# `SENTIMENTAL ANALYSIS`

#### `IMPORTIN LIBRARIES`

In [31]:
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### `IMPORTING DATASET`

In [32]:
ds=pd.read_csv("tweet_train.csv",encoding="ISO-8859-1",names=["label", "ids", "date", "flag", "user", "tweet"])
ds=ds.drop(["ids","date","flag","user"],axis=1)

In [33]:
positive=ds[ds["label"]==0][:10000]
neagtive=ds[ds["label"]==4][:10000]

In [34]:
positive.info()
print()
neagtive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   10000 non-null  int64 
 1   tweet   10000 non-null  object
dtypes: int64(1), object(1)
memory usage: 234.4+ KB

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 800000 to 809999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   10000 non-null  int64 
 1   tweet   10000 non-null  object
dtypes: int64(1), object(1)
memory usage: 234.4+ KB


In [35]:
positive

,label,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
9995,0,Aww that's sad
9996,0,stupid dvds stuffing up the good bits in jaws.
9997,0,@Dandy_Sephy No. Only close friends and family...
9998,0,CRAP! After looking when I last tweeted... WHY...


In [36]:
neagtive["label"]=neagtive["label"].apply(lambda x:1)
neagtive

,label,tweet
800000,1,I LOVE @Health4UandPets u guys r the best!!
800001,1,im meeting up with one of my besties tonight! ...
800002,1,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800003,1,Being sick can be really cheap when it hurts t...
800004,1,@LovesBrooklyn2 he has that effect on everyone
...,...,...
809995,1,Morning! I have slacked for two days in twitte...
809996,1,@bensummers Isn't that sweet of them.... Altru...
809997,1,"@jakrose Um, milk *fathers* don't have udders...."
809998,1,@zenaweist They could also tweet @BeccaRoberts


In [37]:
dataset=pd.concat([positive,neagtive])
dataset=dataset.sample(frac=1)
dataset

,label,tweet
5313,0,I'm not excited for the yearly Holy Week trip ...
804190,1,is starting her day of laundry and cleaning.. ...
7592,0,@loqenz_niteowl I'm glad she got there safe! M...
326,0,I wish I was better at writing. It's taking me...
804384,1,"Ooh, need a TV! @twitchhiker is on This Mornin..."
...,...,...
803016,1,@RadarDog grass??? And it does not make them s...
4646,0,@ravogd a spider? can it be covered when im ro...
802688,1,Funny kid gives the evil look - Pretty damn fu...
2628,0,hungry dreaming of yumyums..


### `PRE-PROCESSING`

##### `PRE PROCESSING VARIABLE SETTING`

In [38]:
# dataset['tweet'] = dataset['tweet'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
dataset['tweet'] = dataset['tweet'].apply(lambda x: re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+",' ',x))

In [39]:
dataset

,label,tweet
5313,0,I m not excited for the yearly Holy Week trip ...
804190,1,is starting her day of laundry and cleaning le...
7592,0,I m glad she got there safe Miss her already
326,0,I wish I was better at writing It s taking me ...
804384,1,Ooh need a TV twitchhiker is on This Morning a...
...,...,...
803016,1,grass And it does not make them sick I would...
4646,0,a spider can it be covered when im round sli...
802688,1,Funny kid gives the evil look Pretty damn funny
2628,0,hungry dreaming of yumyums


In [40]:
dataset["tweet"]=dataset["tweet"].apply(lambda x: " ".join([w for w in x.split() if w not in stopwords.words("english")]))
dataset

,label,tweet
5313,0,I excited yearly Holy Week trip beach
804190,1,starting day laundry cleaning leaving Bath T 2...
7592,0,I glad got safe Miss already
326,0,I wish I better writing It taking long write p...
804384,1,Ooh need TV twitchhiker This Morning 11ish Big...
...,...,...
803016,1,grass And make sick I would like two leg picki...
4646,0,spider covered im round slight phobia
802688,1,Funny kid gives evil look Pretty damn funny
2628,0,hungry dreaming yumyums


In [41]:
# Stemming
st=PorterStemmer()
dataset["tweet"]=dataset['tweet'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [42]:
# Lemmatizing

dataset['tweet']=dataset['tweet'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
dataset

,label,tweet
5313,0,i excit yearli holi week trip beach
804190,1,start day laundri clean leav bath t 20 hour al...
7592,0,i glad got safe miss alreadi
326,0,i wish i better write it take long write paper
804384,1,ooh need tv twitchhik thi morn 11ish big stuff...
...,...,...
803016,1,grass and make sick i would like two leg pick ...
4646,0,spider cover im round slight phobia
802688,1,funni kid give evil look pretti damn funni
2628,0,hungri dream yumyum


### `SPLITTING DATA FOR TRAINING AND TESTING`

#### `LSTM MODEL`

In [43]:
max_words = 1000
max_len=50

def tokenize_pad_sequences(text):
    tokenizer = Tokenizer(num_words=max_words, lower=True, split=' ')
    tokenizer.fit_on_texts(text)
    X = tokenizer.texts_to_sequences(text)
    X = pad_sequences(X, padding='post', maxlen=max_len)
    return X, tokenizer

print('Before Tokenization & Padding \n', dataset['tweet'][0])
X, tokenizer = tokenize_pad_sequences(dataset['tweet'])
print('After Tokenization & Padding \n', X[0])

Before Tokenization & Padding 
 awww bummer you shoulda got david carr third day d
After Tokenization & Padding 
 [  1 177  57 419 568   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [44]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_words, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 128)           128000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 50, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 383,194
Trainable params: 383,194
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
from sklearn.model_selection import train_test_split
Y=pd.get_dummies(dataset["label"]).values
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(16000, 50) (16000, 2)
(4000, 50) (4000, 2)


In [46]:
batch_size = 32
model.fit(X_train, y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
500/500 - 61s - loss: 0.6941 - accuracy: 0.5011 - 61s/epoch - 122ms/step
Epoch 2/7
500/500 - 48s - loss: 0.6934 - accuracy: 0.5042 - 48s/epoch - 96ms/step
Epoch 3/7
500/500 - 48s - loss: 0.6934 - accuracy: 0.4983 - 48s/epoch - 96ms/step
Epoch 4/7
500/500 - 61s - loss: 0.6934 - accuracy: 0.5041 - 61s/epoch - 121ms/step
Epoch 5/7
500/500 - 67s - loss: 0.6933 - accuracy: 0.4972 - 67s/epoch - 134ms/step
Epoch 6/7
500/500 - 59s - loss: 0.6934 - accuracy: 0.4911 - 59s/epoch - 118ms/step
Epoch 7/7
500/500 - 71s - loss: 0.6933 - accuracy: 0.4924 - 71s/epoch - 142ms/step


In [51]:
twt = [""]
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=50, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
1/1 - 0s - 34ms/epoch - 34ms/step
positive


In [52]:
model.evaluate(X_test,y_test)

125/125 [==============================] - 6s 27ms/step - loss: 0.6932 - accuracy: 0.4988


[0.6931515336036682, 0.4987500011920929]

In [55]:
print(X_test[0],y_test[0][0])

[  1  77 173 483 173 312 189   1 101  20 483   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0] 1
